In [2]:
# importing the requests library 
import requests
from lxml import html
import os
import glob
import pandas as pd
import time
import numpy as np
from threading import Thread
import queue

In [3]:
URL = "https://twitter.com/user/status/"

data = pd.read_csv('C:/Users/Loris/Documents/Uni/USDA/project/Data/tweets_stance_sentiment_1outof4.csv', sep='~', header=0)

In [4]:
data.head()

,ID,user_id,t_sentiment,t_stance
0,682715594110689280,259896187,positive,remain
1,682715872205651968,6312362,neutral,other
2,682716086681382912,259896187,negative,remain
3,682716257402109952,6312362,positive,other
4,682716531470544896,259896187,neutral,other


In [54]:
result = []

In [56]:
concurrent = 200

def doWork():
    while True:
        row = q.get()
        id = row['ID']
        user_id = row['user_id']
        t_sentiment = row['t_sentiment']
        t_stance = row['t_stance']
        # sending get request and saving the response as response object 
        page = requests.get(URL+str(id))
        tree = html.fromstring(page.content)
        tweetText = tree.xpath('//title/text()')[0]
        split = page.url.split('/')
        tag = split[3]
        if tag=="user":
            tag=""
        if(str(tweetText).startswith('Twitter')):
            tweetText = ""
        else:
            tweetText = " ".join(tweetText.splitlines())
        result.append({'ID': id, 'user_id': user_id, 'user_tag': tag, 't_sentiment': t_sentiment, 't_stance': t_stance, 'text': tweetText})
        q.task_done()

In [57]:
startTime = time.time()
q = queue.Queue(concurrent * 2)
for i in range(concurrent):
    t = Thread(target=doWork)
    t.daemon = True
    t.start()
try:
    for index, row in data.head(100).iterrows():
        q.put(row)
    q.join()
except KeyboardInterrupt:
    sys.exit(1)
    
print("Execution time: ",time.time()-startTime)

Execution time:  1.520341396331787


In [59]:
dataFrame = pd.DataFrame(result)
dataFrame.to_csv('C:/Users/Loris/Documents/Uni/USDA/project/Data/withText_threadtest.csv', index=False)